In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
import numpy as np

# Import API key
sys.path.append("..")
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("WeatherPy_Vacation.csv")
vacation_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,3,Jucas,BR,76.66,broken clouds,-6.5253,-39.5275,Hotel Casa Bella
1,4,Rikitea,PF,74.93,light rain,-23.1203,-134.9692,People ThankYou
2,6,Timbiqui,CO,78.49,light rain,2.7717,-77.6654,Hotel Sharey
3,8,Waingapu,ID,77.31,moderate rain,-9.6567,120.2641,Pondok Wisata Elim
4,13,Kuliyapitiya,LK,75.63,overcast clouds,7.4688,80.0401,Rest House
5,15,Bobon,PH,76.78,overcast clouds,12.5246,124.5636,PATILAN RESIDENCE
6,22,Jamestown,US,76.01,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
7,29,Albany,US,76.46,overcast clouds,42.6001,-73.9662,NaN
8,31,Guerrero Negro,MX,76.03,clear sky,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
9,32,Bansbaria,IN,78.69,scattered clouds,22.9719,88.3994,The Kalyani Guest House


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [4]:
# 4a. Add a marker layer for each city to the map.
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=max_temp,
                                 dissipating=False,
                                 max_intensity=300, 
                                 point_radius=4)

marker_layer = gmaps.marker_layer(locations, 
                                  info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start=vacation_df.loc[vacation_df["City"]=="Jucas"]
vacation_end = vacation_df.loc[vacation_df["City"]=="Jucas"]
vacation_stop1 = vacation_df.loc[vacation_df["City"]=="Rikitea"]
vacation_stop2 = vacation_df.loc[vacation_df["City"]=="Timbiqui"] 
vacation_stop3 = vacation_df.loc[vacation_df["City"]=="Waingapu"] 

In [6]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start[["Lat","Lng"]].to_numpy()[0]
end = vacation_end[["Lat","Lng"]].to_numpy()[0]
stop1 = vacation_stop1[["Lat","Lng"]].to_numpy()[0]
stop2 = vacation_stop2[["Lat","Lng"]].to_numpy()[0]
stop3 = vacation_stop3[["Lat","Lng"]].to_numpy()[0]

In [7]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".

fig = gmaps.figure()

directions_layer = gmaps.directions_layer(start,
                                          end,
                                          waypoints=[stop1,stop2,stop3],
                                          travel_mode="DRIVING" or "BICYCLING" or "WALKING")

fig.add_layer(directions_layer)

#Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start,vacation_end,vacation_stop1,vacation_stop2,vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,3,Jucas,BR,76.66,broken clouds,-6.5253,-39.5275,Hotel Casa Bella
1,3,Jucas,BR,76.66,broken clouds,-6.5253,-39.5275,Hotel Casa Bella
2,4,Rikitea,PF,74.93,light rain,-23.1203,-134.9692,People ThankYou
3,6,Timbiqui,CO,78.49,light rain,2.7717,-77.6654,Hotel Sharey
4,8,Waingapu,ID,77.31,moderate rain,-9.6567,120.2641,Pondok Wisata Elim


In [9]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [10]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure()

locations = itinerary_df[["Lat", "Lng"]]
max_temp = itinerary_df["Max Temp"]

marker_layer = gmaps.marker_layer(
    locations, 
    info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))